In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# Métrica de evaluación
def RMSLE(actual, pred):
    actualLog = np.log(actual + 1)
    predLog = np.log(pred + 1)
    return (np.mean((actualLog - predLog) ** 2)) **.5

In [ ]:
# Se usaran los sets normalizados bajo columnas SOLAMENTE.
train = pd.read_csv('../normalized2_train.csv')
test = pd.read_csv('../normalized2_test.csv')

# Casteamos todo a int, esto es porque XgBoost dio problemas usando el set de datos con los datos heredados.
const = ['antiguedad', 'banos', 'garages', 'metroscubiertos', 'metrostotales', 'metros']
every_column_except_y= [col for col in train.columns if col not in ['preciomt2', 'precio','id', 'logpreciomt2', 'fecha', 'descripcion', 'titulo', 'logprecio']]
columns = [col for col in train[every_column_except_y].columns if col not in const]
# columns
train.loc[:, columns] = train[columns].fillna(0)
for column in columns:
    train[column] = train[column].astype('int32')

In [ ]:
# Generamos el modelo y un set de test para tener alguna metrica
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 10, reg_alpha = 1.2, n_estimators = 600, reg_lambda = 1.4, subsample=0.8, seed=30)
every_column_except_y= [col for col in train.columns if col not in ['preciomt2', 'precio','id', 'logpreciomt2', 'fecha', 'descripcion', 'titulo', 'logprecio']]

X, y = train.loc[:,every_column_except_y], train['logprecio']
#Entrenamos con el dataset entero
xg_reg.fit(train[every_column_except_y], train['logpreciomt2'])

In [ ]:
#Predecimos y guardamos
pred = xg_reg.predict(test[every_column_except_y])
final = test[['id']]
final.loc[:, 'target'] = test.metros * np.expm1(pred)
final.set_index('id').to_csv('../prediction_xgboostTomTHT3.csv')